In [1]:
import numpy as np
from zeros import *

def OstarO(A, n=7):
    return (A.T @ A).round(n)

In [2]:
m = 5 # Number of measurements
k = 2 # Rank of O^*O

## Diagonalized prior 
prior = np.eye(m) * 100
prior[0,0] = 0.1
prior[1,1] = 0.2
prior[2,2] = 4

## The posterior precision at the first k eigenvectors 
precision = m/k + np.mean(np.diag(prior)[:k])


# M = O^*O
M = np.zeros((m,m))
for i in range(k):
    M[i,i] = precision - prior[i,i]

## Find O such that O^*O = M and O has unit norm rows
O = get_A(M, m).T
OstarO(O)

## Since O[2,:] = -O[4,:] we have clusterization

array([[2.55, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 2.45, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ]])

# Clustered


In [59]:
m = 5
k = 2

o1o2 = get_A(np.diag([0.55, 1.45]), k).T

O = np.zeros((m,k))
O[:2,:2] = o1o2
O[2,1] = 1
O[3,0] = 1
O[4,0] = 1

# Returns O^*O
OstarO(O)

array([[ 2.55, -0.  ],
       [-0.  ,  2.45]])

# Not Clustered


In [3]:
m = 5
k = 2

o1o2 = get_A(np.diag([0.75, 1.25]), k).T
o3o4 = get_A(np.diag([0.8, 1.2]), k).T

O = np.zeros((m,k))
O[:2,:2] = o1o2 
O[2:4,:2] = o3o4
O[4,0] = 1

# Returns O^*O
print(OstarO(O))

# Looks nicer
O = np.array([
    [np.sqrt(3/8), -np.sqrt(5/8)],
    [np.sqrt(3/8),  np.sqrt(5/8)],
    [np.sqrt(2/5), -np.sqrt(3/5)],
    [np.sqrt(2/5),  np.sqrt(3/5)],
    [1,             0           ]        
])

# Returns O^*O
print(OstarO(O))

[[2.55 0.  ]
 [0.   2.45]]
[[2.55 0.  ]
 [0.   2.45]]
